In [1]:
import pandas as pd
from itertools import chain, combinations

In [2]:
transactions = pd.read_excel("../data/transactions.xlsx")

In [3]:
def get_items(dataframe):
    all_items = []
    for x in dataframe.dropna().iloc[0].values:
        str_x = str(x)
        if str_x.endswith('.0'):
            str_x = str_x[:-2]
        all_items.append(str_x)
    return set(all_items)

In [4]:
items = get_items(transactions)
print(items)

{'84950', '22169', '84991', '21866', '23183', '82580', '22903', '20961', '22577', '22246', '22953', '22621', '22071', '22179', '84970S', '22131', '22476', '22377', '22666', '23158', '79321', '21056', '21422', '21070', '22721', '85014B', '21030', '21888', '22367', '21948', '21055', '22144', '22617', '84951B', '21791', '85053', '20675', '22792', '22578', '22614', '21932', '21915', '85039A', '22701', '90166', '22107', '21890', '21786', '84997D', '23309', '22581', '23266', '20718', '22573', '22589', '22174', '22138', '21984', '21156', '23218', '22195', '84945', '23224', '84077', '21025', '85054', '22082', '21212', '22961', '84969', '21126', '21466', '22098', '84375', '22338', 'CRUK', '20677', '20707', '21098', '22467', '22686', '20765', '23084', '21508', '21992', '23197', '22786', '20725', '22508', '21975', '16238', '22326', '85176', '22446', '22736', '35648', '22362', '22070', '22170', '22173', '23223', '23284', '23275', '22557', '22439', '47310M', '23192', '21730', '21704', '85034B', '21

In [5]:
from collections import Counter, defaultdict

def above_min_supp(itemSet, item_counts, minSup, globalItemSetWithSup):
    freqItemSet = set()
    for item in itemSet:
        if item in item_counts:
            
            globalItemSetWithSup[item] += item_counts[item]
            support = float(item_counts[item] / len(item_counts) * 100) 
            if(support >= minSup):
                freqItemSet.add(frozenset((item,)))

    return freqItemSet

In [6]:
def union(items, length):
    return set([i.union(j) for i in items for j in items if len(i.union(j)) == length])


In [7]:
item_counts = Counter(transactions.values.flatten().astype(str))

KeyboardInterrupt: 

In [92]:

item_set_with_count = defaultdict(int)
items_with_supp = above_min_supp(items, item_counts, 20, item_set_with_count)

In [8]:
def prun(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet

In [9]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [24]:
def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            print(item)
            break
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
        break
    return rules

In [27]:
def aprio(dataset, minSup, minConf, item_counts):
    
    C1ItemSet = get_items(dataset)
    
    globalFreqItemSet = dict()
    
    globalItemSetWithSup = defaultdict(int)

    #item_counts = Counter(transactions.values.flatten().astype(str))

    L1ItemSet = above_min_supp(C1ItemSet, item_counts, minSup, globalItemSetWithSup)

    currentLSet = L1ItemSet

    k = 2

    # Calculating frequent item set
    while(currentLSet):
        
        globalFreqItemSet[k-1] = currentLSet
        
        candidateSet = union(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = prun(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = above_min_supp(candidateSet, item_counts, minSup, globalItemSetWithSup)
        k += 1


    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

In [26]:
item_counts = Counter(transactions.values.flatten().astype(str))

In [28]:

globalFreqItemSet, rules = aprio(transactions, 20, 1, item_counts)

frozenset({'23084'})


In [19]:
rules

[]